# The Self-Consistency Pattern - A "Generate-and-Synthesize" Approach

Hello everyone. We've seen how Chain-of-Thought can produce a single, high-quality reasoning path. Today, we're exploring a more advanced application of the **Self-Consistency** pattern, moving beyond a simple "majority vote" to a powerful "generate-and-synthesize" workflow.

The core idea is a two-step process:

1. **Diverge (Generate):** First, we ask the model to generate multiple, diverse solutions to a creative problem.
2. **Converge (Synthesize):** Then, we feed all those solutions into a second, "editor" prompt that synthesizes them into a single, superior final output.

This mimics a real-world creative team: a brainstorming session followed by an editing session. We'll demonstrate this by generating high-quality documentation for a Python function.

## The Task and Helper Functions

Our task is to generate a comprehensive docstring for a Python function that is complex enough to benefit from detailed documentation. It takes multiple arguments and has a clear return value and potential error cases.

In [ ]:
import litellm
import inspect
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    if "gpt-5" in model:
        kwargs["max_completion_tokens"] = max_tokens
    else:
        kwargs["max_tokens"] = max_tokens
        
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    return litellm.completion(
        model=model,
        messages=parsed_messages,
        **kwargs
    )

# --- The Context: A More Complex Function to Document ---
def parse_log_entry(entry_line: str, strict_mode: bool = True) -> dict | None:
    """
    Example format: "ERROR|2024-07-27T10:00:00Z|User login failed"
    """
    parts = entry_line.split('|', 2)
    if len(parts) != 3:
        if strict_mode:
            raise ValueError("Log entry must have 3 parts separated by vertical bars.")
        return None

    return {
        "level": parts[0],
        "timestamp": parts[1],
        "message": parts[2]
    }

CODE_TO_DOCUMENT = inspect.getsource(parse_log_entry)

print("Setup complete. Function context is ready.")

## The Generation Phase (Divergence)

First, we will ask the model to generate three different docstring drafts. We'll use a single prompt but set `n=3` and a non-zero `temperature` to encourage slightly different wording, focus, and examples in each draft.

In [ ]:
generation_prompt = [
    {
        "role": "system",
        "content": dedent("""
        You are an expert technical writer specializing in Python. Your task is to generate a high-quality
        docstring for a given function.
        """)
    },
    {
        "role": "user",
        "content": dedent(f"""
        Please generate a comprehensive docstring for the following Python function.

        ## Rules
        * Follow the Google Python Style Guide.
        * Include sections for summary, Args, Returns, and Raises.
        * Keep it succint.

        ## Code
        ```python
        {CODE_TO_DOCUMENT}
        ```
        """)
    }
]

response = get_completion(
    generation_prompt,
    temperature=1.3,
    n=3
)

In [ ]:
drafts_docstrings = []

print("--- Generated Drafts ---")
for i, choice in enumerate(response.choices):
    draft = choice.message.content
    drafts_docstrings.append(draft)
    print(f"\n>>> Draft {i + 1} <<<")
    print(draft)

## The Synthesis Phase (Convergence)

Now for the powerful part. We will feed all three drafts into a new prompt. This prompt uses an "Editor" persona whose only job is to combine the best elements of each draft into a single, polished, final version.